# Time-frequency analysis
This notebook re-analyzes and extracts power from the 
resting state???

- which method to use: Hilbert-filter, complex morlet wavelet convolution, Short-time FFT (Welch), Multitapers
- If we use Welch method what is the size of the fft window? should we have zero paddings? should we have overlap? what to do for the edge artefacts?
- Normalization: which method to use? Z transform, percentage change or decibels? which part of the recording should be used as the baseline? which baseline window is the best?

In [39]:
# setup
import mne
import numpy as np
import pandas as pd
import os.path as op
import matplotlib.pyplot as plt
import mne_bids
from autoreject import get_rejection_threshold
from preprocessing import _epochs_to_continuous, _make_montage
from xarray_creator import _cut_noisy

# open preprocessed data
n_sub = '03'
task = 'induction1'
con_dict = {}
data_dir = '/Users/yeganeh/Codes/otka-preprocessing/data/Main-study/derivatives/mne-bids-pipeline'
epoch_name = f'sub-{n_sub}_ses-01_task-{task}_proc-clean_epo.fif'
raw_name = f'sub-{n_sub}_ses-01_task-{task}_proc-filt_raw.fif'

dir = op.join(data_dir, f'sub-{n_sub}/ses-01/eeg/{raw_name}')

# open clean epochs
# epoch = mne.read_epochs(dir)
raw = mne.io.read_raw(dir)



Opening raw data file /Users/yeganeh/Codes/otka-preprocessing/data/Main-study/derivatives/mne-bids-pipeline/sub-03/ses-01/eeg/sub-03_ses-01_task-induction1_proc-filt_raw.fif...
Isotrak not found
    Range : 0 ... 363259 =      0.000 ...   363.259 secs
Ready.


In [34]:
from mne.time_frequency import psd_welch
psd = psd_welch(raw, n_fft=1051, n_per_seg=1051, fmax=40)

Effective window size : 1.051 (s)


## The impact of different parameters on welch method
Below image shows is that if we use epoched data and then average over the epoch dimention is the same as we concatenate epochs and then calculate the psd using Welch method.

In [ ]:
bids_path = mne_bids.BIDSPath(subject='01', session='01', task='baseline1', root='data/BIDS_data')
raw = mne_bids.read_raw_bids(bids_path, verbose=False)
raw = _cut_noisy(raw, 'baseline1', 'hun')
raw.load_data().filter(0.5, 42, h_trans_bandwidth='auto')
pos = _make_montage()
raw.set_montage(pos)

epochs = mne.make_fixed_length_epochs(raw, duration=1)
reject = get_rejection_threshold(epochs)
epochs.drop_bad(reject=reject)
data = _epochs_to_continuous(epochs)
info = mne.create_info(ch_names=raw.ch_names, sfreq=1000, ch_types='eeg')
raw2 = mne.io.RawArray(data, info)
raw2.set_channel_types({'ECG':'ecg',
                       'EOG1':'eog',
                       'EOG2':'eog'})
raw2.set_montage(pos)
raw2.pick_types(eeg=True)

In [ ]:
# Estimate PSDs based on "mean" and "median" averaging for comparison.
from mne.time_frequency import psd_welch
kwargs = dict(fmin=2, fmax=40, n_jobs=-1, n_fft=1000, n_overlap=0)
psds_welch_epoch, freqs_mean = psd_welch(epochs, **kwargs)
psds_welch_epoch = psds_welch_epoch.mean(0)
psds_welch_concatenated, freqs_median = psd_welch(raw2, **kwargs)

# Convert power to dB scale.
psds_welch_epoch = 10 * np.log10(psds_welch_epoch)
psds_welch_concatenated = 10 * np.log10(psds_welch_concatenated)

# We will only plot the PSD for a single sensor in the first epoch.
ch_name = 'O1'
ch_idx = raw.info['ch_names'].index(ch_name)

_, ax = plt.subplots()
ax.plot(freqs_mean, psds_welch_epoch[ch_idx, :], color='k',
        ls='-', label='RAW')
ax.plot(freqs_median, psds_welch_concatenated[ch_idx, :], color='k',
        ls='--', label='CONCATENATED')

ax.set(title='Welch PSD ({})'.format(ch_name),
       xlabel='Frequency (Hz)', ylabel='Power Spectral Density (dB)')
ax.legend(loc='upper right')
plt.show()